# **Imports**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Load the formated data**

In [3]:
directory = '/content/drive/My Drive/Colab Notebooks/North_Sea/'
exec(open(directory + 'src/inputs_rdn.py').read())

In [4]:
time_years, inp_true_test, chl_stand, NAO_stand, MLD_stand, SST_stand, wind_stand, AMO_stand = var_stand(directory)

# **Method 1**

> ## **Load the model**

In [5]:
params = {}
params['dim_chloro'] = 1
params['dim_phy'] = 5
params['dim_input_vect'] = params['dim_chloro'] + params['dim_phy']
params['dim_input'] = params['dim_input_vect']
params['dim_hidden_1'] = 32
params['dim_hidden_2'] = 28
params['dim_hidden_3'] = 14
params['dim_output'] = params['dim_input']

In [6]:
exec(open(directory + 'src/RNN.py').read())
model_RNN_1 = INT_net(params)
model_RNN_1.load_state_dict(torch.load(directory + 'models/model_1_weights.pth'))

<All keys matched successfully>

> ## **Variables strenght assessment**

In [7]:
## SEE HOW THE ERROR EVOLVES WHEN RANDOMIZING ONE GIVEN PHYSICAL VARIABLE
variables = ['NAO', 'MLD', 'SST', 'wind', 'AMO']
Chl_stand_test = inp_true_test[:,0].reshape(1,-1)[0]
var_test = np.var(Chl_stand_test)
model_RNN_1.double()
for i, var in enumerate(variables):
    print("[%d/%d] : "%(i+1, len(variables)), var)
    MSE_table = []
    for k in range(100):
        inp_test_rdn, inp_true_test_rdn, dt_test = inputs_rdn_1(time_years, chl_stand, NAO_stand, MLD_stand, SST_stand, wind_stand, AMO_stand, randomized_variable = var)
        NN_outputs_test = []
        old_pred = torch.from_numpy(inp_test_rdn[0]).view(1,-1)
        for i in range(len(inp_test_rdn)):
            pred = model_RNN_1(old_pred, torch.from_numpy(dt_test[i]).view(1,-1))
            old_pred = torch.cat((pred[0,0].view(1,-1), torch.from_numpy(inp_true_test_rdn)[i,1:].view(1,-1)), axis = 1)
            NN_outputs_test.append(pred.cpu().detach().numpy()[0][0])

        NMSE = (Chl_stand_test - NN_outputs_test)**2/var_test
        MSE_table.append(np.mean(NMSE))
    print("Mean MSE :", np.mean(MSE_table))
    print("STD  MSE :", np.std(MSE_table))
    print()

[1/5] :  NAO
Mean MSE : 0.9626670135107463
STD  MSE : 0.009373489780027527

[2/5] :  MLD
Mean MSE : 1.7372003530435274
STD  MSE : 0.12574243032994836

[3/5] :  SST
Mean MSE : 2.910153535006516
STD  MSE : 0.17310888888915038

[4/5] :  wind
Mean MSE : 1.0138815234150464
STD  MSE : 0.025664644854645667

[5/5] :  AMO
Mean MSE : 0.7880413610584802
STD  MSE : 0.012813359094589017



>> **Variables classification :**
>> 1.   SST
>> 2.   MLD
>> 3.   wind
>> 4.   NAO
>> 5.   AMO



# **Method 2**

> ## **Load the model**

In [8]:
params['dt_horizon'] = 6

In [9]:
model_RNN_2 = INT_net(params)
model_RNN_2.load_state_dict(torch.load(directory + 'models/model_2_weights.pth'))

<All keys matched successfully>

> ## **Variables strenght assessment**

In [10]:
## SEE HOW THE ERROR EVOLVES WHEN RANDOMIZING ONE GIVEN PHYSICAL VARIABLE
variables = ['NAO', 'MLD', 'SST', 'wind', 'AMO']
Chl_stand_test = inp_true_test[:,0].reshape(1,-1)[0]
var_test = np.var(Chl_stand_test)
model_RNN_2.double()
for i, var in enumerate(variables):
    print("[%d/%d] : "%(i+1, len(variables)), var)
    MSE_table = []
    for k in range(100):
        inp_test_rdn, inp_true_test_rdn, dt_test = inputs_rdn_1(time_years, chl_stand, NAO_stand, MLD_stand, SST_stand, wind_stand, AMO_stand, randomized_variable = var)
        NN_outputs_test = []
        old_pred = torch.from_numpy(inp_test_rdn[0]).view(1,-1)
        for i in range(len(inp_test_rdn)):
            pred = model_RNN_2(old_pred, torch.from_numpy(dt_test[i]).view(1,-1))
            old_pred = torch.cat((pred[0,0].view(1,-1), torch.from_numpy(inp_true_test_rdn)[i,1:].view(1,-1)), axis = 1)
            NN_outputs_test.append(pred.cpu().detach().numpy()[0][0])

        NMSE = (Chl_stand_test - NN_outputs_test)**2/var_test
        MSE_table.append(np.mean(NMSE))
    print("Mean MSE :", np.mean(MSE_table))
    print("STD  MSE :", np.std(MSE_table))
    print()

[1/5] :  NAO
Mean MSE : 0.5534531570222059
STD  MSE : 0.009987374105847418

[2/5] :  MLD
Mean MSE : 0.9263130205577512
STD  MSE : 0.05239600653848514

[3/5] :  SST
Mean MSE : 2.7446571948021528
STD  MSE : 0.20522906880145378

[4/5] :  wind
Mean MSE : 0.7945572543496924
STD  MSE : 0.042086157921875166

[5/5] :  AMO
Mean MSE : 0.5085156826984302
STD  MSE : 0.003334789117037108



>> **Variables classification :**
>> 1.   SST
>> 2.   wind
>> 3.   MLD
>> 4.   NAO
>> 5.   AMO



# **Method 3**

> ## **Load the model**

In [11]:
params['dt_horizon'] = 4
params['dim_input_vect'] = params['dim_chloro'] + params['dim_phy']
params['dim_input'] = params['dim_input_vect'] * params['dt_horizon'] 
params['dim_hidden_1'] = 200
params['dim_hidden_2'] = 150
params['dim_hidden_3'] = 100
params['dim_output'] = params['dim_input']

In [12]:
model_RNN_3 = INT_net(params)
model_RNN_3.load_state_dict(torch.load(directory + 'models/model_3_weights.pth'))

<All keys matched successfully>

> ## **Variables strenght assessment**

In [13]:
## SEE HOW THE ERROR EVOLVES WHEN RANDOMIZING ONE GIVEN PHYSICAL VARIABLE
variables = ['NAO', 'MLD', 'SST', 'wind', 'AMO']
Chl_stand_test = inp_true_test[:,0].reshape(1,-1)[0]
var_test = np.var(Chl_stand_test)
for i, var in enumerate(variables):
    print("[%d/%d] : "%(i+1, len(variables)), var)
    MSE_table = []
    for k in range(100):
        inp_test_rdn, inp_true_test_rdn, dt_test = inputs_rdn_3(time_years, chl_stand, NAO_stand, MLD_stand, SST_stand, wind_stand, AMO_stand, params['dt_horizon'], randomized_variable = var)
        NN_outputs_test = []
        old_pred = inp_test_rdn[0].view(1,-1)
        for i in range(len(inp_test_rdn)):
            six_prev_months = old_pred
            pred = model_RNN_3(old_pred, torch.ones([1,1]).view(1,-1))
            old_pred = torch.cat((six_prev_months[0, params['dim_input_vect']:].view(1,-1), pred[0,0].view(1,-1)), axis = 1) # drop the oldest month (t-6) of lenght params['dim_input_vect'] = 6, and concat with Chl predicted at month t
            old_pred = torch.cat((old_pred, inp_true_test_rdn[i,1:params['dim_input_vect']].view(1,-1)), axis = 1) # concat with the true physics à month t
            NN_outputs_test.append(pred.cpu().detach().numpy()[0,0])

        NMSE = (Chl_stand_test[params['dt_horizon']-1:-params['dt_horizon']] - NN_outputs_test)**2/var_test
        MSE_table.append(np.mean(NMSE))
    print("Mean MSE :", np.mean(MSE_table))
    print("STD  MSE :", np.std(MSE_table))
    print()

[1/5] :  NAO
Mean MSE : 0.6075584091766857
STD  MSE : 0.011725939249347929

[2/5] :  MLD
Mean MSE : 0.8402855991078672
STD  MSE : 0.034350607733785385

[3/5] :  SST
Mean MSE : 1.875814047976614
STD  MSE : 0.1355805944092045

[4/5] :  wind
Mean MSE : 0.6609942693868822
STD  MSE : 0.020423049731748202

[5/5] :  AMO
Mean MSE : 0.6142014171131945
STD  MSE : 0.0023597814420204906



>> **Variables classification :**
>> 1.   SST
>> 2.   wind
>> 3.   MLD
>> 4.   NAO
>> 5.   AMO

